In [111]:
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense, Conv1D, Reshape
from tensorflow.keras.models import Sequential

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ingma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
qa = pd.read_json("data/lemmatize_google_qa.json.xz")
jokes = pd.read_json("data/lemmatize_jokes.json.xz")
articles = pd.read_json("data/lemmatize_news.json.xz")

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(["...", "'s", "wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d", "ha", "´´", "´", "´´´", ",", "!", "'", ":", ";", '"', "\\", "``"])
stop_words.extend(["wa", "''", "the", "he", "my", "it"])

data_for_df = []

jokes_lemmas = jokes['lemmatize']

for i, joke in enumerate(jokes_lemmas):
    joke_words = [word for word in joke if word not in stop_words and word.isalpha()]
    joke_text = " ".join(joke_words)
    data_for_df.append({'text': joke_text, 'joke': 1, 'category': 0})
    
qa_lemmas = qa['lemmatize']

for i, q in enumerate(qa_lemmas):
    qa_words = [word for word in q if word not in stop_words and word.isalpha()]
    qa_text = " ".join(qa_words)
    data_for_df.append({'text': qa_text, 'joke': 0, 'category': 1})
    
articles_lemmas = articles['lemmatize']

for i, article in enumerate(articles_lemmas):
    article_words = [word for word in article if word not in stop_words and word.isalpha()]
    article_text = " ".join(article_words)
    article_text = re.sub("reporting .*", "", article_text)
    data_for_df.append({'text': article_text, 'joke': 0, 'category': 2})

In [7]:
df = pd.DataFrame(data_for_df)
df.shape

(1380351, 3)

In [8]:
df = df.drop_duplicates('text')
df.shape

(1283149, 3)

In [12]:
df[df.joke == 0].head(5)
df_ = df
df = pd.concat([df_[df_.joke == 0].sample(frac=0.04), df_[df_.joke == 1].sample(frac=0.04)])

### Preparing data for Neural Network

In [13]:
from sklearn.model_selection import train_test_split

In [24]:
X, X_test, y, y_test = train_test_split(df['text'], df['joke'], test_size=0.1, stratify=df['joke'])

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [27]:
num_words = 15000
maxlen = 500

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

tokenized_X_train = tokenizer.texts_to_sequences(X_train)
tokenized_X_val = tokenizer.texts_to_sequences(X_val)
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(tokenized_X_train, maxlen=maxlen)
X_val_pad = pad_sequences(tokenized_X_val, maxlen=maxlen)
X_test_pad = pad_sequences(tokenized_X_test, maxlen=maxlen)

In [28]:
y_train_vec = to_categorical(y_train)
y_val_vec = to_categorical(y_val)
y_test_vec = to_categorical(y_test)

In [29]:
X_train_pad.shape

(41573, 500)

In [30]:
X_test_pad.shape

(5133, 500)

In [31]:
X_val_pad.shape

(4620, 500)

### Neural Network training and testing

In [130]:
# "model.add(Embedding(len(tokenizer.word_index), 32, input_length=maxlen))\n",
#     "model.add(Conv1D(256, 10, activation='relu'))\n",
#     "model.add(Flatten())\n",
#     "model.add(Dense(10, activation='softmax'))\n",
#     "model.add(Flatten())\n",
#     "model.add(Dense(1, activation='sigmoid'))\n"
import tensorflow as tf
model = Sequential()
model.add(Reshape((20, 25), dtype=tf.float32))
model.add(Conv1D(256, 10, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [131]:
import numpy as np
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train_pad.astype(np.float32), y_train, validation_data=(X_val_pad, y_val), epochs=10, verbose=1, batch_size=256, workers=8, use_multiprocessing=True)
model.evaluate(X_test_pad, y_test)

Epoch 1/10
163/163 [==============================] - 3s 15ms/step - loss: 0.6030 - acc: 0.7591 - val_loss: 0.5841 - val_acc: 0.7695

[0.4975481331348419, 0.8007013201713562]

In [45]:
model.save("convulational_100k_embedding.h5")

In [97]:
from tensorflow.keras.models import load_model
new_model = load_model('model_with_embedding_layer.h5')

In [103]:
tokenizer.word_index.keys()

dict_keys(['said', 'one', 'say', 'year', 'man', 'state', 'n', 'first', 'new', 'would', 'day', 'time', 'go', 'like', 'w', 'two', 'get', 'united', 'km', 'mi', 'also', 'sq', 'last', 'back', 'company', 'people', 'doe', 'could', 'million', 'come', 'woman', 'percent', 'well', 'world', 'told', 'take', 'know', 'see', 'may', 'make', 'call', 'three', 'wife', 'guy', 'second', 'country', 'week', 'joke', 'asked', 'got', 'american', 'city', 'many', 'since', 'season', 'home', 'president', 'government', 'next', 'trump', 'right', 'month', 'want', 'old', 'game', 'look', 'billion', 'house', 'group', 'bank', 'asks', 'good', 'little', 'around', 'market', 'part', 'name', 'made', 'way', 'tell', 'going', 'work', 'think', 'friend', 'show', 'best', 'national', 'walk', 'china', 'still', 'start', 'south', 'song', 'u', 'life', 'point', 'end', 'went', 'doctor', 'later', 'film', 'called', 'head', 'need', 'much', 'north', 'give', 'son', 'long', 'thing', 'night', 'even', 'never', 'including', 'used', 'number', 'bar', 

In [106]:
import io
weights = model.get_layer(index=0).get_weights()[0]
out_v = io.open('vectors_cnn.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata_cnn.tsv', 'w', encoding='utf-8')

for index, word in enumerate(list(tokenizer.word_index.keys())):
  if index == 0: continue # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()